### Exhaustive Feature Selection

**Exhaustive Feature Selection** finds the **best subset of features** of all possible feature subsets, according to a **determined performance metric** for a certain machine learning algorithm. For example, if we train a **logistic regression** and the dataset consists of **4 features**, the algorithm will evaluate all **15** feature combinations as follows: All possible combinations of 1 feature! All possible combinations of 2 features! All possible combinations of 3 features! All the 4 features! And select the one that results in the **best performance** (e.g., classification accuracy) of the logistic regression.

Exhaustive Feature Selection is a **greedy algorithm** as **it evaluates all possible feature combinations**. It is **very computationally expensive**, and sometimes, if feature space is big, **even unfeasible.**

There is a special package for python that implements this type of feature selection: mlxtend. http://rasbt.github.io/mlxtend/. In the mlxtend implementation of the Exhaustive Feature Selection, the **stopping criteria** is an arbitrarily set number of features. So the search will finish when we reach the desired number of selected features. 

This is somewhat arbitrary, we might be selecting a **sub-opimal number of features**, or likewise, a high number of features. But, by looking at the performance metric returned by the algorithm as it selects the features, we can have a view, if more features do add value, or not. 

**Note** If we wanted to stop the search by using another criteria, we would have to code the algorithm ourselves, unfortunately :(Here I will use the Step Forward feature selection algorithm from mlxtend in a classification and regression dataset.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

## Classification

**Load a few rows to from dataset to speed things up!**

In [3]:
data = pd.read_csv('dataset_2.csv', nrows=10000)
data.shape

(10000, 109)

In [4]:
data.head()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.532710,3.280834,17.982476,4.404259,2.349910,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832,0.976806,2.541417
1,5.821374,12.098722,13.309151,4.125599,1.045386,1.832035,1.833494,0.709090,8.652883,0.102757,...,2.479789,7.795290,3.557890,17.383378,15.193423,8.263673,1.878108,0.567939,1.018818,1.416433
2,1.938776,7.952752,0.972671,3.459267,1.935782,0.621463,2.338139,0.344948,9.937850,11.691283,...,1.861487,6.130886,3.401064,15.850471,14.620599,6.849776,1.098210,1.959183,1.575493,1.857893
3,6.020690,9.900544,17.869637,4.366715,1.973693,2.026012,2.853025,0.674847,11.816859,0.011151,...,1.340944,7.240058,2.417235,15.194609,13.553772,7.229971,0.835158,2.234482,0.946170,2.700606
4,3.909506,10.576516,0.934191,3.419572,1.871438,3.340811,1.868282,0.439865,13.585620,1.153366,...,2.738095,6.565509,4.341414,15.893832,11.929787,6.954033,1.853364,0.511027,2.599562,0.811364


**Important**

In all feature selection procedures, it is good practice to **select the features by examining only the training set** to **avoid overfit.**

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

((7000, 108), (3000, 108))

### Remove correlated features

The Exhaustive Feature Selection takes a long time to run, so to speed it up we will reduce the feature space by removing correlated features first.

In [6]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
corr_features = correlation(X_train, 0.6)  # reduce the correlation threshold remove more features!
print('correlated features: ', len(set(corr_features)) )

correlated features:  59


**Removed correlated features!**

In [7]:
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((7000, 49), (3000, 49))

###  Exhaustive Feature Selection

For the **Exhaustive Feature Selection algorithm**, Use the class **EFS from MLXtend:** http://rasbt.github.io/mlxtend/user_guide/feature_selection/ExhaustiveFeatureSelector/

**Try all possible 1 and 2 feature combinations to shorter search time!** In a **multicore or distributed computer
system** you can try **more greedy searches**!

Within the EFS we indicate: the algorithm we want to create (here RandomForests, few trees to speed things up), the number of minimum  and maximum features we want our model to have! The evaluation metric: in this case the roc_auc! The cross-validation! This is going to take a while, do not despair**

In [9]:
efs = EFS(RandomForestClassifier(n_estimators=5,
                                 n_jobs=4,
                                 random_state=0,
                                 max_depth=2),
          min_features=1,
          max_features=2,
          scoring='roc_auc',
          print_progress=True,
          cv=2)
efs = efs.fit(np.array(X_train), y_train)  # search features

Features: 1225/1225

The log above means that the search evaluated 1225 feature combinations!

In [10]:
efs.best_idx_

(12, 32)

In [11]:
selected_feat = X_train.columns[list(efs.best_idx_)]
selected_feat

Index(['var_16', 'var_55'], dtype='object')

### Compare performance of feature subsets

**Function to train random forests and evaluate the performance!**

In [12]:
def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

**Evaluate performance of classifier using selected features!**

In [13]:
run_randomForests(X_train[selected_feat],
                  X_test[selected_feat],
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.7152616074556793
Test set
Random Forests roc-auc: 0.6868327155531925


**For comparison, we train random forests using all features (except the correlated ones, which we removed already)!**

In [14]:
run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.7509464508443697
Test set
Random Forests roc-auc: 0.6874840935262856


Even with 2 features, **the performance is not super far off** of that model built using all features!!

## Regression

Let's now repeat the process but in the context of regression. With the house prices dataset from Kaggle, the aim is to predict the continuous target: House Price.

In [35]:
data = pd.read_csv('HousingPrices_train.csv')
data.shape

(1460, 81)

In practice, feature selection should be done **after data pre-processing**, so ideally, all **the categorical variables are encoded into numbers**, and then you can assess how deterministic they are of the target!** Here for simplicity use only numerical variables! **Select numerical columns:**

In [36]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

((1022, 37), (438, 37))

### Remove correlated features

In [38]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
corr_features = correlation(X_train, 0.6)  # Reduce the threshold to remove more features!
print('correlated features: ', len(set(corr_features)) )

correlated features:  9


**Removed correlated  features!**

In [39]:
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((1022, 28), (438, 28))

In [40]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

###  Exhaustive Feature Selection

Try all possible **10 and 11 feature combinations** to shorter search time!** In a **multicore or distributed computer** system you can try **more greedy searches!**

In [41]:
efs = EFS(RandomForestRegressor(n_estimators=5,
                                n_jobs=4,
                                random_state=0,
                                max_depth=2),
          min_features=1,
          max_features=2,
          scoring='r2',
          print_progress=True,
          cv=2)
efs = efs.fit(np.array(X_train), y_train)

Features: 406/406

**The searched evaluated 406 feature combinations!**

In [42]:
efs.best_idx_

(4, 9)

In [43]:
X_train.columns[list(efs.best_idx_)]

Index(['OverallQual', 'BsmtFinSF1'], dtype='object')

### Compare performance of feature subsets

**function to train random forests and evaluate the performance!**

In [44]:
def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict(X_train)
    print('Random Forests roc-auc: {}'.format(r2_score(y_train, pred)))
    print('Test set')
    pred = rf.predict(X_test)
    print('Random Forests roc-auc: {}'.format(r2_score(y_test, pred)))

In [45]:
selected_feat = X_train.columns[list(efs.best_idx_)]

**Evaluate performance of algorithm built, using selected features!**

In [46]:
run_randomForests(X_train[selected_feat],
                  X_test[selected_feat],
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.7598962603686706
Test set
Random Forests roc-auc: 0.6856450799849569


**For comparison, we train random forests using all features (except the correlated ones, which we removed already)!**

In [47]:
run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.8454047044087043
Test set
Random Forests roc-auc: 0.7806708079073277


In this case, the **performance decreased quite a lot**, so there are **additional features that are good predictors** of the target. **The Exhaustive Search** is **extremely computationally expensive**. Use rarely this selection procedure for this same reason. But if you have access to **super computers**, you may give it a go.